In [8]:
import os
import random
import warnings

import pandas as pd
import spacy
from dotenv import load_dotenv
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.neighbors import NearestNeighbors
from sqlalchemy import create_engine

load_dotenv()
warnings.filterwarnings("ignore")

In [9]:
POSTGRES_ADDRESS = os.getenv("POSTGRES_ADDRESS")
POSTGRES_PORT = os.getenv("POSTGRES_PORT")
POSTGRES_USERNAME = os.getenv("POSTGRES_USERNAME")
POSTGRES_PASSWORD = os.getenv("POSTGRES_PASSWORD")
POSTGRES_DBNAME = os.getenv("POSTGRES_DBNAME")

postgres_str = ('postgresql://{username}:{password}@{ipaddress}:{port}/{dbname}'
                .format(username=POSTGRES_USERNAME,
                        password=POSTGRES_PASSWORD,
                        ipaddress=POSTGRES_ADDRESS,
                        port=POSTGRES_PORT,
                        dbname=POSTGRES_DBNAME))

cnx = create_engine(postgres_str)

data = pd.read_sql_query("SELECT * FROM get_book_data", cnx)
data.fillna("missing", inplace=True)
data.shape

(58, 8)

In [10]:
list(data.columns)

['id',
 'book_title',
 'publisher_name',
 'category_1',
 'category_2',
 'category_3',
 'author_1',
 'author_2']

In [11]:
nlp = spacy.blank("vi")
nlp.Defaults.stop_words.add("sách")


def clean_text(text):
    doc = nlp(text.lower())
    tokens = [token.text for token in doc if not token.is_stop and not token.is_punct]
    return " ".join(tokens)


data["clean_text"] = (
        data["book_title"] + " " +
        data["publisher_name"] + " " +
        data["category_1"] + " " +
        data["category_2"] + " " +
        data["category_3"] + " " +
        data["author_1"] + " " +
        data["author_2"]
)

data["clean_text"] = data["clean_text"].apply(clean_text)

data.iloc[random.choices(range(len(data)), k=10)]

,id,book_title,publisher_name,category_1,category_2,category_3,author_1,author_2,clean_text
35,98,Cái tết của mèo con,NXB Trẻ,Tiểu thuyết,Trong nước,missing,Ngô Mạnh Lân,Nguyễn Đình Thi,tết mèo nxb trẻ tiểu thuyết missing ngô lân ng...
38,101,Naruto - Tập 64,NXB Kim Đồng,Giả tưởng,Manga,missing,Masashi Kishimoto,missing,naruto tập 64 nxb kim đồng giả tưởng manga mis...
30,93,Tiểu Thuyết Naruto - Sasuke Chân Truyền,NXB Kim Đồng,Giả tưởng,Light novel,Nước ngoài,Masashi Kishimoto,Shin Towada,tiểu thuyết naruto sasuke chân truyền nxb kim ...
48,112,Tiểu thuyết Naruto - Itachi chân truyền - Quan...,NXB Kim Đồng,Giả tưởng,Nước ngoài,Tiểu thuyết,Masashi Kishimoto,missing,tiểu thuyết naruto itachi chân truyền quang mi...
20,83,Sách Giáo Khoa Hoá Học Lớp 10,Giáo Dục Việt Nam,Sách giáo khoa,missing,missing,missing,missing,giáo khoa hoá học lớp 10 giáo dục việt nam g...
39,102,Naruto - Tập 50,NXB Kim Đồng,Giả tưởng,Manga,missing,Masashi Kishimoto,missing,naruto tập 50 nxb kim đồng giả tưởng manga mis...
9,70,Thám tử lừng danh Conan - Tập 103,NXB Kim Đồng,Manga,Trinh thám,missing,missing,missing,thám tử lừng danh conan tập 103 nxb kim đồng m...
47,111,Tiểu thuyết Naruto - Itachi chân truyền - Ám d...,NXB Kim Đồng,Giả tưởng,Nước ngoài,Tiểu thuyết,Masashi Kishimoto,missing,tiểu thuyết naruto itachi chân truyền ám thiên...
38,101,Naruto - Tập 64,NXB Kim Đồng,Giả tưởng,Manga,missing,Masashi Kishimoto,missing,naruto tập 64 nxb kim đồng giả tưởng manga mis...
36,99,Thơ Nguyễn Khuyến,NXB Tổng Hợp TPHCM,Tiểu thuyết,Trong nước,missing,Nguyễn Khuyến,missing,thơ nguyễn khuyến nxb tổng hợp tphcm tiểu th...


In [12]:
cv = CountVectorizer()
clean_text_vector = cv.fit_transform(data['clean_text']).toarray()
model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(clean_text_vector)

NearestNeighbors(algorithm='brute', metric='cosine')

In [13]:
def recommend(text, k=5):
    cleaned_text = cv.transform([clean_text(text)]).toarray()
    distances, indices = model.kneighbors(cleaned_text, n_neighbors=k)
    # return data.iloc[indices[0]][["id", "book_title"]].to_dict(orient="records")
    return data[["book_title", "publisher_name", "category_1", "category_2", "category_3", "author_1", "author_2"]].iloc[indices[0]]


In [15]:
user_description = "tiểu thuyết của nxb kim đồng"
recommend(user_description)

,book_title,publisher_name,category_1,category_2,category_3,author_1,author_2
47,Tiểu thuyết Naruto - Itachi chân truyền - Ám d...,NXB Kim Đồng,Giả tưởng,Nước ngoài,Tiểu thuyết,Masashi Kishimoto,missing
48,Tiểu thuyết Naruto - Itachi chân truyền - Quan...,NXB Kim Đồng,Giả tưởng,Nước ngoài,Tiểu thuyết,Masashi Kishimoto,missing
51,Gambit Hậu,NXB Kim Đồng,Nước ngoài,Tiểu thuyết,missing,Walter Tevis,missing
57,The rake get ravished,NXB Kim Đồng,Lãng mạn,Nước ngoài,Tiểu thuyết,Sophie Jordan,missing
24,Viên ngọc trai kì diệu,NXB Kim Đồng,Nước ngoài,Tiểu thuyết,missing,Elisa Sabatinelli,missing
